In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
dataset=dataset.drop("User ID",axis=1)

In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [12]:
from sklearn.neighbors import KNeighborsClassifier


In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors':np.arange(1,11),
              'weights': ['uniform','distance'],
              'algorithm': ['auto','ball_tree','kd_tree','brute'],
               'p':[1,2]} 

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
   
# fitting the model for grid search 
grid.fit(X_train_, y_train) 
 


Fitting 5 folds for each of 160 candidates, totalling 800 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             scoring='f1', verbose=3)

In [14]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test_) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)



In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'algorithm': 'auto', 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}: 0.9183922682195829


In [16]:
print("The confusion Matrix:\n",cm)
        

The confusion Matrix:
 [[78  7]
 [ 4 45]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.5

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005105,0.004366,0.015155,0.001969,auto,1,1,uniform,"{'algorithm': 'auto', 'n_neighbors': 1, 'p': 1...",0.764706,0.800000,0.772727,0.809524,0.833333,0.796058,0.024954,145
1,0.004194,0.002137,0.008250,0.001311,auto,1,1,distance,"{'algorithm': 'auto', 'n_neighbors': 1, 'p': 1...",0.764706,0.800000,0.772727,0.809524,0.833333,0.796058,0.024954,145
2,0.002610,0.000970,0.012460,0.001502,auto,1,2,uniform,"{'algorithm': 'auto', 'n_neighbors': 1, 'p': 2...",0.764706,0.800000,0.772727,0.750000,0.888889,0.795264,0.049555,149
3,0.002935,0.000978,0.005909,0.000588,auto,1,2,distance,"{'algorithm': 'auto', 'n_neighbors': 1, 'p': 2...",0.764706,0.800000,0.772727,0.750000,0.888889,0.795264,0.049555,149
4,0.002200,0.000401,0.012293,0.000742,auto,2,1,uniform,"{'algorithm': 'auto', 'n_neighbors': 2, 'p': 1...",0.666667,0.727273,0.769231,0.764706,0.774194,0.740414,0.040430,153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.000806,0.000987,0.004281,0.001158,brute,9,2,distance,"{'algorithm': 'brute', 'n_neighbors': 9, 'p': ...",0.764706,0.833333,0.809524,0.926829,0.974359,0.861750,0.077290,21
156,0.001041,0.000869,0.003821,0.000188,brute,10,1,uniform,"{'algorithm': 'brute', 'n_neighbors': 10, 'p':...",0.764706,0.833333,0.789474,0.950000,0.918919,0.851286,0.072053,65
157,0.000612,0.000809,0.003951,0.000737,brute,10,1,distance,"{'algorithm': 'brute', 'n_neighbors': 10, 'p':...",0.764706,0.864865,0.780488,0.926829,0.918919,0.851161,0.067785,69
158,0.000806,0.000987,0.003630,0.001506,brute,10,2,uniform,"{'algorithm': 'brute', 'n_neighbors': 10, 'p':...",0.764706,0.833333,0.789474,0.926829,0.947368,0.852342,0.072900,61


In [21]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Gender Male 0 or 1:"))

Age: 28
Salary: 20000
Gender Male 0 or 1: 1


In [22]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
